In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model

# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann.h5')
ann_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               256256    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 297473 (1.13 MB)
Trainable params: 297473 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import json

# Define the path to your JSON file
file_path = '/content/drive/MyDrive/MasterThesis/captions_data_combined_all.json'

# Open and load the JSON data
with open(file_path, 'r') as file:
    captions_data = json.load(file)


In [ ]:
import torch
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms
from torchvision.models import resnet50
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
from tensorflow.keras.models import load_model
from nltk.tokenize import sent_tokenize, word_tokenize
import json
from tqdm import tqdm

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann.h5')

# Pre-trained ResNet50 model for feature extraction
resnet_model = resnet50(pretrained=True)
resnet_model.eval()

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            return img
        else:
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def extract_features(image, model):
    img_t = transform(image).unsqueeze(0)  # Apply transformation with augmentation
    with torch.no_grad():
        features = model(img_t)
    return features.numpy().flatten()

# Function to select key phrases from text
def select_key_phrases(text, num_words=3):
    words = word_tokenize(text)
    # Select the first few words based on num_words
    selected_words = ' '.join(words[:num_words])
    return selected_words

# Function to create combined text for summarization
def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
    if gpt2_weight > blip_weight:
        primary_caption = gpt2_caption
        secondary_caption = select_key_phrases(blip_caption, 3)  # Take only a few words from the less weighted caption
    else:
        primary_caption = blip_caption
        secondary_caption = select_key_phrases(gpt2_caption, 3)  # Take only a few words from the less weighted caption

    combined_text = primary_caption + ' ' + secondary_caption
    return combined_text

for key, item in tqdm(captions_data.items()):
    img_url = item['url']
    img = download_image(img_url)
    if img is not None:
        features = extract_features(img, resnet_model).reshape(1, -1)
        predicted_prob = ann_model.predict(features)
        predicted_label = (predicted_prob > 0.5).astype(int)

        print(predicted_label)
        # Determine weights based on prediction
        gpt2_weight, blip_weight = (2, 1) if predicted_label == 0 else (1, 2)

        gpt2_caption = item['generated_captions'].get('gpt2', '')
        blip_caption = item['generated_captions'].get('blip', '')

        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)

        input_length = len(combined_text.split())
        max_length_ratio = 0.75
        min_viable_length = 20
        max_length = max(min_viable_length, int(input_length * max_length_ratio))
        min_length = max(10, max_length // 2)

        summarized_caption = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        item['summarized_caption'] = summarized_caption
        item['most_similar_caption'] = most_similar_caption
        item['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)


output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_ann.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
  0%|          | 0/1090 [00:00<?, ?it/s]

1/1 [==============================] - 0s 263ms/step


Your max_length is set to 20, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


[[0]]


  0%|          | 1/1090 [00:05<1:43:28,  5.70s/it]

...

Summarized Caption:  a man standing in a kitchen with a dog a dog is a kitchen . a man
Most Similar Original Caption: A man points at the wall in his kitchen.
Cosine Similarity Score: 0.7744706273078918
1/1 [==============================] - 0s 22ms/step


Your max_length is set to 20, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


[[0]]


100%|██████████| 1090/1090 [1:12:33<00:00,  3.99s/it]

Summarized Caption:  A bathroom with a sink, mirror, and shower is a bathroom without a sink or
Most Similar Original Caption: A bathroom with a sink, vanity and shower stall.
Cosine Similarity Score: 0.9087074995040894
JSON file has been updated and saved.


In [ ]:
#!pip install torch torchvision transformers nltk tensorflow keras language_tool_python
import torch
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms
from torchvision.models import resnet50
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
from tensorflow.keras.models import load_model
from nltk.tokenize import sent_tokenize, word_tokenize
import json
from tqdm import tqdm

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann.h5')

# Pre-trained ResNet50 model for feature extraction
resnet_model = resnet50(pretrained=True)
resnet_model.eval()

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            return img
        else:
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def extract_features(image, model):
    img_t = transform(image).unsqueeze(0)  # Apply transformation with augmentation
    with torch.no_grad():
        features = model(img_t)
    return features.numpy().flatten()

# Function to create combined text for summarization
def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
  combined_text = (gpt2_caption + ' ') * gpt2_weight + (blip_caption + ' ') * blip_weight
  return combined_text

for key, item in tqdm(captions_data.items()):
    img_url = item['url']
    img = download_image(img_url)
    if img is not None:
        features = extract_features(img, resnet_model).reshape(1, -1)
        predicted_prob = ann_model.predict(features)
        predicted_label = (predicted_prob > 0.5).astype(int)

        print(predicted_label)
        # Determine weights based on prediction
        gpt2_weight, blip_weight = (2, 1) if predicted_label == 0 else (1, 2)

        gpt2_caption = item['generated_captions'].get('gpt2', '')
        blip_caption = item['generated_captions'].get('blip', '')

        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)
        summarized_caption = summarizer(combined_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        item['summarized_caption'] = summarized_caption
        item['most_similar_caption'] = most_similar_caption
        item['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_summary_modified.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/1090 [00:00<?, ?it/s]

1/1 [==============================] - 0s 329ms/step


Your max_length is set to 130, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


[[0]]


  0%|          | 1/1090 [00:12<3:42:26, 12.26s/it]

...

Summarized Caption:  a kitchen with a stove, a sink, and a refrigerator  a man standing in a kitchen  with a dog a dog . a kitchen without a refrigerator is a kitchen full of a refrigerator, a stove and a sink .
Most Similar Original Caption: A man points at the wall in his kitchen.
Cosine Similarity Score: 0.7565215229988098
1/1 [==============================] - 0s 26ms/step


Your max_length is set to 130, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


[[0]]


100%|██████████| 1090/1090 [2:23:43<00:00,  7.91s/it]

Summarized Caption:  A bathroom with a sink, mirror, and shower in it is a bathroom without a mirror . A bathroom has a mirror in it can be found in a bathroom that has a sink and mirror .
Most Similar Original Caption: A bathroom with a sink, vanity and shower stall.
Cosine Similarity Score: 0.8648346066474915
JSON file has been updated and saved.


In [ ]:
import torch
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms
from torchvision.models import resnet50
from transformers import CLIPProcessor, CLIPModel, pipeline
import nltk
from tensorflow.keras.models import load_model
from nltk.tokenize import sent_tokenize, word_tokenize
import json
from tqdm import tqdm

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the summarization pipeline and CLIP components
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Load the trained ANN model
ann_model = load_model('/content/drive/MyDrive/MasterThesis/model_ann.h5')

# Pre-trained ResNet50 model for feature extraction
resnet_model = resnet50(pretrained=True)
resnet_model.eval()

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            return img
        else:
            return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

def extract_features(image, model):
    img_t = transform(image).unsqueeze(0)  # Apply transformation with augmentation
    with torch.no_grad():
        features = model(img_t)
    return features.numpy().flatten()

# Function to select key phrases from text
def select_key_phrases(text, num_words=3):
    words = word_tokenize(text)
    # Select the first few words based on num_words
    selected_words = ' '.join(words[:num_words])
    return selected_words

# Function to create combined text for summarization
def create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight):
    if gpt2_weight > blip_weight:
        primary_caption = gpt2_caption

    else:
        primary_caption = blip_caption


    combined_text = primary_caption
    return combined_text

for key, item in tqdm(captions_data.items()):
    img_url = item['url']
    img = download_image(img_url)
    if img is not None:
        features = extract_features(img, resnet_model).reshape(1, -1)
        predicted_prob = ann_model.predict(features)
        predicted_label = (predicted_prob > 0.5).astype(int)

        print(predicted_label)
        # Determine weights based on prediction
        gpt2_weight, blip_weight = (2, 1) if predicted_label == 0 else (1, 2)

        gpt2_caption = item['generated_captions'].get('gpt2', '')
        blip_caption = item['generated_captions'].get('blip', '')

        combined_text = create_combined_text_for_summarization(gpt2_caption, blip_caption, gpt2_weight, blip_weight)

        input_length = len(combined_text.split())
        max_length_ratio = 0.75
        min_viable_length = 20
        max_length = max(min_viable_length, int(input_length * max_length_ratio))
        min_length = max(10, max_length // 2)

        summarized_caption = summarizer(combined_text, max_length=max_length, min_length=min_length, do_sample=False)[0]['summary_text']

        # Encode the summarized caption and original captions to get embeddings
        inputs = clip_processor(text=[summarized_caption] + item['original_coco_captions'], return_tensors="pt", truncation=True, padding=True)
        embeddings = clip_model.get_text_features(**inputs)

        # Calculate cosine similarity between the summarized caption and original captions
        similarities = [torch.cosine_similarity(embeddings[0], emb, dim=0).cpu().item() for emb in embeddings[1:]]
        most_similar_index = similarities.index(max(similarities))
        most_similar_caption = item['original_coco_captions'][most_similar_index]
        cosine_similarity_score = max(similarities)

        # Update the JSON structure with new information
        item['summarized_caption'] = summarized_caption
        item['most_similar_caption'] = most_similar_caption
        item['cosine_similarity_score'] = cosine_similarity_score

        print(f"Summarized Caption: {summarized_caption}")
        print(f"Most Similar Original Caption: {most_similar_caption}")
        print(f"Cosine Similarity Score: {cosine_similarity_score}")
        print("=" * 50)

# Specify the output path for the updated JSON data
output_path = '/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_only_blip_gpt.json'  # Update this path as needed

# Save the modified JSON data to a new file
with open(output_path, 'w') as file:
    json.dump(captions_data, file, indent=4)

print("JSON file has been updated and saved.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Summarized Caption:  a man standing in a kitchen with a dog is a man with a woman with a
Most Similar Original Caption: A man points at the wall in his kitchen.
Cosine Similarity Score: 0.737080991268158
1/1 [==============================] - 0s 20ms/step


Your max_length is set to 20, but your input_length is only 13. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


[[0]]


100%|██████████| 1090/1090 [57:18<00:00,  3.15s/it]

Summarized Caption:  A bathroom with a sink, mirror, and shower  is a bathroom with sink,
Most Similar Original Caption: A bathroom with a sink, vanity and shower stall.
Cosine Similarity Score: 0.9136794805526733
JSON file has been updated and saved.


In [ ]:
import requests
from PIL import Image
from io import BytesIO
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import numpy as np

# Transformation pipeline with data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def download_image(url):
    """ Download an image from a URL and return a PIL image. """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return Image.open(BytesIO(response.content))
        else:
            print(f"Failed to download image: HTTP {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
    return None

def show_transformations(image, transform):
    """ Apply transformations step by step to an image and display each step. """
    fig, axs = plt.subplots(1, 5, figsize=(15, 3))
    titles = ['Original', 'Resize', 'CenterCrop', 'ToTensor', 'Normalize']
    images = [image]

    # Temporary variable to hold transformation stages
    temp_image = image

    # Apply each transform individually and append to list for visualization
    for t, title in zip(transform.transforms, titles):
        if isinstance(temp_image, torch.Tensor):
            temp_image = t(temp_image)
            if title == 'Normalize':
                # Ensure tensor is in (C, H, W) format expected for normalization
                temp_image = temp_image.clone().detach()
                temp_image = temp_image * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
                temp_image = temp_image.clamp(0, 1)
        else:
            temp_image = t(temp_image)

        # Convert tensor for display
        if isinstance(temp_image, torch.Tensor):
            display_image = temp_image.permute(1, 2, 0).numpy()
            display_image = np.clip(display_image, 0, 1)
        else:
            display_image = temp_image

        images.append(display_image)

    # Display each transformation
    for ax, img, title in zip(axs, images, titles):
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')
    plt.show()

i=0
# Assuming `captions_data` is a loaded JSON containing your dataset
for key, item in tqdm(captions_data.items()):
    image = download_image(item['url'])
    show_transformations(image, transform)
    print(item['original_coco_captions'])
    i+=1
    if i==20:
      break;


Output hidden; open in https://colab.research.google.com to view.